# --------------------------------------1、导入所需要的包--------------------------------------------

In [4]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models

from scipy import io
import pandas as pd
import os
from PIL import Image
import pickle
import re
import import_ipynb

import numpy as np
from model_resnet18 import resnet18

importing Jupyter notebook from model_resnet18.ipynb


# --------------------------------------2、准备数据集--------------------------------------------------
- 之前已经准备好了数据集的预处理和裁剪

In [2]:
"""创建自定义数据类"""
# 返回一个元组，包含图片及其对应的标签
class Dog(data.Dataset):
    def __init__(self, data_dir, train_dict, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.train_dict = train_dict
        self.file_list = os.listdir(data_dir) # 获取数据集中所有文件的列表
    
    def __len__(self):
        return len(self.file_list) # 返回数据集中样本长度
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.file_list[idx]) # 获取图片的完整路径
        image = Image.open(img_name).convert('RGB')  # 读取图像并转换为RGB格式
        
        # 判断是否使用transform函数
        if self.transform:
            image = self.transform(image)
        
        label = self.train_dict[self.file_list[idx]]
        label -= 1
        label = torch.tensor(label, dtype=torch.long)
        return image, label # 返回经过处理的图像文件和标签
        
        

In [3]:
"""定义图片增广操作"""
# 这一步很关键，针对小数据集模型很容易过拟合！！！需要数据增广来实现数据扩大化
transform_train = torchvision.transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08～1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224x224的新图像
    torchvision.transforms.RandomResizedCrop(224, scale=(0.08, 1.0),
                                             ratio=(3.0/4.0, 4.0/3.0)),
    torchvision.transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    torchvision.transforms.ColorJitter(brightness=0.4,
                                       contrast=0.4,
                                       saturation=0.4),
    # 添加随机噪声
    torchvision.transforms.ToTensor(),
    # 标准化图像的每个通道
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])

transform_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])])

In [5]:
"""实例化数据集"""

data_dir = 'data/resize_2/train'
with open('data/resize_2/train_dict.pickle', 'rb') as pickle_file:
    loaded_dict = pickle.load(pickle_file)

train_data = Dog(data_dir, loaded_dict, transform=transform_train)  

In [6]:
"""打包用于训练的数据集"""
batch_size = 32
train_data_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0 )

# --------------------------------------3、构建ResNet18网络模型---------------------------------

- 先构建残差块
- 然后逐层搭建模型
- 实例化模型

In [5]:
"""定义使用GPU进行计算"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model_train = resnet18(120)
model_train = model_train.to(device)

In [7]:
# 查看模型可学习的参数量
total_params = 0
# 遍历模型的所有参数，并累加它们的元素数量
for param in model_train.parameters():
    # 参数元素数量等于其形状中的元素个数之积
    num_params = param.numel()
    total_params += num_params

print(f"Total parameters: {total_params}")

Total parameters: 11241080



# --------------------------------------4、定义损失函数与优化器----------------------------------

- 使用agd作为优化器
- 使用交叉熵作为损失函数

In [28]:
"""定义交叉熵函数"""
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_train.parameters(), lr=0.001, weight_decay=0.0001)

# --------------------------------------6、定义训练函数----------------------------------------------

- 都能自动保存最优模型
- 显示每个epoch的训练损失和验证损失

In [30]:
"""定义训练函数"""
def train(train_data_loader, num_epoch):
    
    for epoch in range(num_epoch):
        
        train_loss = []
        train_accs = []
        best_acc= 0.0
        
        for image, label in train_data_loader:
            
            # 将批量数据导入到GPU中
            image = image.to(device)
            label = label.to(device)
            
            # 前向传播
            outputs = model_train(image) 
            loss = criterion(outputs, label)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # 计算小批量的损失和准确度
            _, preds = torch.max(outputs, 1) # 选择最大的作为预测结果
            acc = (preds == label).float().mean() # 此处返回每个小批量下的准确性，如0.5的准确率
            train_loss.append(loss.item())         
       
            train_accs.append(acc)
        
        # 记录每个epoch的损失和准确性
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # 打印每个epoch的loss和acc
        print(f"Train：第{epoch + 1}/{num_epoch}个循环，损失为：{train_loss:.5f}, 准确度为：{train_acc:.5f}")
        
        # 保存模型
        if train_acc > best_acc:
            best_acc = train_acc
            torch.save(model_train.state_dict(), model_path)
            print('saveing model with acc {:.3f}'.format(best_acc))
            

# --------------------------------------7、训练模型----------------------------------------------------

In [31]:
num_epoch = 10
model_path = 'best2.ckpt'
train_iter = train_data_loader
train(train_iter, num_epoch)

Train：第1/10个循环，损失为：2.36720, 准确度为：0.38825
saveing model with acc 0.388
Train：第2/10个循环，损失为：2.32512, 准确度为：0.39817
saveing model with acc 0.398
Train：第3/10个循环，损失为：2.31250, 准确度为：0.40033
saveing model with acc 0.400
Train：第4/10个循环，损失为：2.26998, 准确度为：0.40292
saveing model with acc 0.403
Train：第5/10个循环，损失为：2.25041, 准确度为：0.41750
saveing model with acc 0.417
Train：第6/10个循环，损失为：2.22863, 准确度为：0.42017
saveing model with acc 0.420
Train：第7/10个循环，损失为：2.21914, 准确度为：0.42342
saveing model with acc 0.423
Train：第8/10个循环，损失为：2.16282, 准确度为：0.43158
saveing model with acc 0.432
Train：第9/10个循环，损失为：2.17778, 准确度为：0.43175
saveing model with acc 0.432
Train：第10/10个循环，损失为：2.17644, 准确度为：0.42942
saveing model with acc 0.429


# --------------------------------------8、在测试集上检验模型精度-------------------------------

In [32]:
"""准备验证数据集"""
test_dir = 'data/resize_2/test'
with open('data/resize_2/test_dict.pickle', 'rb') as pickle_file:
    loaded_test_dict = pickle.load(pickle_file)

test_data = Dog(test_dir, loaded_test_dict, transform=transform_test)  
test_data_loader = data.DataLoader(test_data, batch_size=32, shuffle=False)

In [33]:
"""加载模型"""
model_test = resnet18(120)
model_test.load_state_dict(torch.load('best2.ckpt'))
model_test = model_test.to(device)

In [34]:
"""定义衡量准确度方法"""
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct /  total

In [35]:
model_test.eval()  # 设置模型为评估模式（这会关闭dropout和batch normalization等）

with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for images, labels in test_data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model_test(images)
        accuracy = calculate_accuracy(outputs, labels)
        # print(accuracy)
        total_correct += accuracy * images.size(0)
        total_samples += images.size(0)

test_accuracy = total_correct / total_samples
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.4501


# --------------------------------------9、搭建一个GUI来部署模型--------------------------------

- 输入一张图片，输出一个分类

In [39]:
import tkinter as tk
from PIL import ImageTk, Image
import torch
from torchvision import transforms

# 加载模型
model_tk = resnet18(120)  # 加载模型，需要替换为你的加载模型的方法
model_tk.load_state_dict(torch.load('best2.ckpt'))  # 加载模型参数
model_tk.eval()  # 设置模型为评估模式

# 图像预处理
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 定义预测函数
def predict_image():
    image_path = entry.get()  # 获取输入的图片路径
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)  # 添加批次维度
    
    with torch.no_grad():
        output = model_tk(image)
    
    predicted_class = torch.argmax(output, dim=1).item()  # 获取预测的类别索引
    label.config(text=f"Predicted Class: {predicted_class}")  # 在界面输出预测类别

# 创建GUI窗口
root = tk.Tk()
root.title("Image Classifier")

# 创建界面元素
label_path = tk.Label(root, text="Image Path:")
label_path.pack()

entry = tk.Entry(root)
entry.pack()

button = tk.Button(root, text="Predict", command=predict_image)
button.pack()

label = tk.Label(root)
label.pack()

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\miniconda\envs\machine_zzh\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\De\AppData\Local\Temp\ipykernel_16916\3755948913.py", line 21, in predict_image
    image = Image.open(image_path)
  File "D:\miniconda\envs\machine_zzh\lib\site-packages\PIL\Image.py", line 3131, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'a'
